In [2]:
import requests
import time
import pandas as pd
import numpy as np

In [159]:
df = pd.DataFrame({
    "id_vacancy": [],
    "job_vacancy": [],
    "description": [],
    "salary": []
})

In [173]:
params = {
    "area": 113,  # Россия
    "industry": 7,  # https://api.hh.ru/industries // IT-сфера
    "per_page": 50,
}

req = requests.get("https://api.hh.ru/vacancies/", params).json()
pages = req['pages']

In [87]:
vacID = []
for page in range(pages):
    params.update({"page": page})
    request = requests.get("https://api.hh.ru/vacancies/", params)
    vacancies = request.json()['items']
    request.close()
    for vacancy in vacancies:
        vacID.append(vacancy['id'])

In [161]:
vac_ID = np.array(vacID, dtype=int).reshape(4, -1)

In [137]:
def salary(info: dict) -> float:
    f = info['from']
    t = info['to']
    if f and t:
        return (f + t) / 2
    if f is None:
        return t
    return f

In [162]:
def fill_df(vac_ID):
    for id in vac_ID:
        request = requests.get(f"https://api.hh.ru/vacancies/{id}")
        vacancy_info = request.json()
        request.close()
        try:
            job_vacancy = vacancy_info['name']
            description = vacancy_info['description']
            sal = vacancy_info['salary']
            if job_vacancy and description and sal:
                row = {
                    "id_vacancy": id,
                    "job_vacancy": job_vacancy,
                    "description": description,
                    "salary": salary(sal)
                }
                df.loc[len(df)] = row
        except:
            time.sleep(5)

In [163]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor() as executor:
    executor.submit(fill_df, vac_ID[0])
    executor.submit(fill_df, vac_ID[1])
    executor.submit(fill_df, vac_ID[2])
    executor.submit(fill_df, vac_ID[3])

In [167]:
df.to_csv("vacancies.csv", sep='|', index=False)